In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2022-ml-finalexam-p1/sample_submission.csv
/kaggle/input/2022-ml-finalexam-p1/train.csv
/kaggle/input/2022-ml-finalexam-p1/test.csv


문제 설명 : 배틀그라운드 최종 등수 예측
배틀그라운드는 매 경기마다 최대 100명의 플레이어가 배틀로얄 형식으로 서로 싸우면서 살아남는 것을 목표로 하는 게임이다. 본 문제는 배틀그라운드라는 게임 내 데이터를 활용하여 특정 플레이어의 경기 중 행동 정보를 기반으로 해당 플레이어의 최종 등수가 어떻게 될 것인지 예측하는 것을 목표로 한다. 본 문제에서는 최대 4명의 플레이어가 하나의 팀을 이뤄 경기가 진행되는 스쿼드 모드를 고려한다. (하지만 데이터에서는 여러 이유로 4명 이상의 플레이어가 하나의 팀으로 집계된 경우가 존재함)
구체적으로 각 플레이어의 정보는 경기 ID (matchId), 팀 ID (groupId)와 함께 경기 중 수행한 다양한 행동에 대한 정보 (걷거나 뛴 거리 (walkDistance), 죽인 적 플레이어 수 (kills), 적 플레이어에게 가한 데미지량 (damageDealt), 회복 아이템 사용 (heals,boosts) 등)으로 구성되어 있으며, 학습 데이터에는 각 팀 별로 1 (우승)부터 0 (최하위 등수)까지 scaling된 최종 등수 정보 (winPlacePerc)가 포함되어 있다. 따라서 예측 또한 최종 등수에 대한 1부터 0 사이의 scaling된 값으로 이루어져야한다.

In [2]:
train = pd.read_csv('/kaggle/input/2022-ml-finalexam-p1/train.csv')
test= pd.read_csv('/kaggle/input/2022-ml-finalexam-p1/test.csv')
#데이터 불러오기

In [3]:
train #데이터 확인

,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,...,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc
0,023a9418cf67b0,18b16ec699d8b6,0000a43bce5eec,0,0,200.00,2,0,1,21,...,0,0.0,0,0.0,0,0,73.970,1,1474,0.3333
1,6703bc191cff0a,18b16ec699d8b6,0000a43bce5eec,0,1,19.35,0,0,0,61,...,0,0.0,0,0.0,0,0,189.400,1,1484,0.3333
2,5a3afae17b53c0,236ab9e9c081b9,0000a43bce5eec,0,0,66.65,0,0,0,93,...,0,0.0,0,0.0,0,0,17.510,1,1430,0.0370
3,e431d8a8f6c99b,236ab9e9c081b9,0000a43bce5eec,0,0,21.29,0,0,0,92,...,0,0.0,0,0.0,0,0,5.201,1,1503,0.0370
4,e7b325d63d8393,236ab9e9c081b9,0000a43bce5eec,0,0,0.00,0,0,0,91,...,0,0.0,0,0.0,0,0,18.650,1,1450,0.0370
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70243,15f0a85253bee8,ade7512835fe69,0a42f5450ccc91,0,0,19.35,0,0,0,94,...,0,0.0,0,0.0,0,0,20.450,1,0,0.0000
70244,84241448d4ac32,b655c7cf75b15a,0a42f5450ccc91,0,0,0.00,0,0,0,87,...,0,0.0,0,0.0,0,0,86.000,2,0,0.0769
70245,618e4e59a977db,b655c7cf75b15a,0a42f5450ccc91,0,0,0.00,0,0,0,89,...,0,0.0,0,0.0,0,0,6.779,1,0,0.0769
70246,52599ea8fb4992,b655c7cf75b15a,0a42f5450ccc91,0,0,37.73,0,0,0,86,...,0,0.0,0,0.0,0,0,126.400,2,0,0.0769


In [4]:
test#데이터 확인

,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,...,rankPoints,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints
0,94a6853fa03acd,b655c7cf75b15a,0a42f5450ccc91,0,0,100.00,0,1,0,43,...,1428,0,0.0,0,0.0,0,0,106.30,2,0
1,1209697b6692fd,b655c7cf75b15a,0a42f5450ccc91,0,0,0.00,0,0,0,88,...,1440,0,0.0,0,0.0,0,0,21.10,2,0
2,0a5d0fe9d96bee,b655c7cf75b15a,0a42f5450ccc91,0,0,100.00,1,1,0,44,...,1428,0,0.0,0,0.0,0,0,33.59,1,0
3,d739f0784af7d5,b655c7cf75b15a,0a42f5450ccc91,0,0,100.00,2,1,0,40,...,1428,0,0.0,0,0.0,1,0,172.60,1,0
4,0bd7c9d014c3e8,b655c7cf75b15a,0a42f5450ccc91,0,0,0.00,0,0,0,85,...,1442,0,0.0,0,0.0,0,0,183.10,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17556,ec65a03d1cd842,d7d457caee0480,0c9ff6254a7640,0,0,0.00,0,0,0,84,...,-1,0,0.0,0,0.0,0,0,131.30,3,1509
17557,ce24e7b3647325,d7d457caee0480,0c9ff6254a7640,1,0,19.30,0,0,0,81,...,-1,0,0.0,0,0.0,0,0,279.10,2,1472
17558,c8b9a7f6a6e8c1,d7d457caee0480,0c9ff6254a7640,0,0,158.70,0,0,0,85,...,-1,0,0.0,0,0.0,0,0,108.60,2,1515
17559,b414ba9c6fd303,d7d457caee0480,0c9ff6254a7640,0,0,76.78,1,0,0,39,...,-1,0,0.0,0,0.0,0,0,323.20,1,1479


In [5]:
train = train.drop(['Id','groupId','matchId'],axis =1)
test = test.drop(['Id','groupId','matchId'],axis =1)
#쓸모 없는 데이터 드랍

In [6]:
x_train = train.drop(['winPlacePerc'],axis =1)
x_test = test
y_train = train['winPlacePerc']
# drop하고 y_train에 넣기

In [7]:
from sklearn.ensemble import RandomForestRegressor
knn = RandomForestRegressor()
knn.fit(x_train,y_train)
y_pred = knn.predict(x_test)
#일단 값들이 0,1처럼 나뉘는게 아니고 0과1사이에 있으니 회귀를 써야한다. 
#그러나 우리가 배운 함수들중 svr과 knn은 아니다. 따라서 randomForesREsgressor를 사용했다. 

In [8]:
submit = pd.read_csv('/kaggle/input/2022-ml-finalexam-p1/sample_submission.csv')
#submit불러오기

In [9]:
submit['winPlacePerc'] = y_pred
submit = submit.to_csv('submit.csv',index = False)
#y_pred를 저장하고 to_csv를 한다.